In [68]:
import numpy as np

# 1. Policy Iteration
## 1.1 Policy Evaluation

In [77]:
def get_state(state, action):
    
    action_grid = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    state[0]+=action_grid[action][0]
    state[1]+=action_grid[action][1]
    
    if state[0] < 0 :
        state[0] = 0
    elif state[0] > 3 :
        state[0] = 3
    
    if state[1] < 0 :
        state[1] = 0
    elif state[1] > 3 :
        state[1] = 3
    
    return state[0], state[1]

def policy_evaluation(grid_width, grid_height, action, policy, iter_num, reward=-1, dis=1):
    
    # table initialize
    cur_value_table = np.zeros([grid_height, grid_width], dtype=float)
    
    # iteration
    if iter_num == 0:
        print(f'Iteration: {iter_num} \n{cur_value_table}\n')
        return cur_value_table
    
    for iteration in range(iter_num):
        next_value_table = np.zeros([grid_height, grid_width], dtype=float)
        for i in range(grid_height):
            for j in range(grid_width):
                if i == j and ((i == 0) or (i == 3)):
                    value_t = 0
                else :
                    value_t = 0
                    for act in action:
                        i_, j_ = get_state([i,j], act)
                        value = policy[i][j][act] * (reward + dis*cur_value_table[i_][j_])
                        value_t += value
                next_value_table[i][j] = round(value_t, 1)
        print()
        print(f'cur_value_table: {cur_value_table}')
        print(f'next_value_talbe: {next_value_table}')
        print()
                
        iteration += 1
        
        # print result
        if (iteration % 10) != iter_num:
            if iteration > 100:
                if (iteration % 20) == 0:
                    print(f'Iteration: {iteration} \n{next_value_table}\n')
            else:
                if (iteration % 10) == 0:
                    print(f'Iteration: {iteration} \n{next_value_table}\n')
        else:
            print(f'Iteration: {iteration} \n{next_value_table}\n')
        
        cur_value_table = next_value_table
        
    return next_value_table

In [79]:
# env setting

grid_height, grid_width = 4, 4

action = [0, 1, 2, 3] # up, down, left, right


def reset_policy(grid_height, grid_width, action):
    policy = np.empty([grid_height, grid_width, len(action)], dtype=float)
    for i in range(grid_height):
        for j in range(grid_width):
            for k in range(len(action)):
                if i==j and ((i==0) or (i==3)):
                    policy[i][j]=0.00
                else :
                    policy[i][j]=0.25
    policy[0][0] = [0] * grid_width
    policy[3][3] = [0] * grid_width
    return policy

policy = reset_policy(grid_height, grid_width, action)

In [71]:
value = policy_evaluation(grid_height, grid_width, action, policy, 100)


cur_value_table: [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
next_value_talbe: [[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]


cur_value_table: [[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]
next_value_talbe: [[ 0.  -1.8 -2.  -2. ]
 [-1.8 -2.  -2.  -2. ]
 [-2.  -2.  -2.  -1.8]
 [-2.  -2.  -1.8  0. ]]


cur_value_table: [[ 0.  -1.8 -2.  -2. ]
 [-1.8 -2.  -2.  -2. ]
 [-2.  -2.  -2.  -1.8]
 [-2.  -2.  -1.8  0. ]]
next_value_talbe: [[ 0.  -2.4 -3.  -3. ]
 [-2.4 -2.9 -3.  -3. ]
 [-3.  -3.  -2.9 -2.4]
 [-3.  -3.  -2.4  0. ]]


cur_value_table: [[ 0.  -2.4 -3.  -3. ]
 [-2.4 -2.9 -3.  -3. ]
 [-3.  -3.  -2.9 -2.4]
 [-3.  -3.  -2.4  0. ]]
next_value_talbe: [[ 0.  -3.1 -3.8 -4. ]
 [-3.1 -3.7 -4.  -3.8]
 [-3.8 -4.  -3.7 -3.1]
 [-4.  -3.8 -3.1  0. ]]


cur_value_table: [[ 0.  -3.1 -3.8 -4. ]
 [-3.1 -3.7 -4.  -3.8]
 [-3.8 -4.  -3.7 -3.1]
 [-4.  -3.8 -3.1  0. ]]
next_value_talbe: [[ 0.  -3.6 -4.7 -4.9]
 [-3.6 -4.6 -4.8 -4.

## 1.2 Policy Improvement

In [73]:
def policy_improvement(value, action, policy, reward = -1, grid_width = 4):
    
    grid_height = grid_width
    
    action_match = ['Up', 'Down', 'Left', 'Right']
    action_table = []
    
    # get Q-func.
    for i in range(grid_height):
        for j in range(grid_width):
            q_func_list=[]
            if i==j and ((i==0)or (i==3)):
                action_table.append('T')
            else:
                for k in range(len(action)):
                    i_, j_ = get_state([i, j], k)
                    q_func_list.append(value[i_][j_])
                max_actions = [action_v for action_v, x in enumerate(q_func_list) if x == max(q_func_list)] 

                # update policy
                policy[i][j]= [0]*len(action) # initialize q-func_list
                for y in max_actions :
                    policy[i][j][y] = (1 / len(max_actions))

                # get action
                idx = np.argmax(policy[i][j])
                action_table.append(action_match[idx])
    action_table=np.asarray(action_table).reshape((grid_height, grid_width))                
    
    
    print(f'Updated policy is :\n{policy}\n')
    print(f'at each state, chosen action is :\n{action_table}')
    
    return policy

In [72]:
# previous policy
policy

array([[[0.  , 0.  , 0.  , 0.  ],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.  , 0.  , 0.  , 0.  ]]])

In [74]:
updated_policy = policy_improvement(value, action, policy)

Updated policy is :
[[[0.  0.  0.  0. ]
  [0.  0.  1.  0. ]
  [0.  0.  1.  0. ]
  [0.  0.5 0.5 0. ]]

 [[1.  0.  0.  0. ]
  [0.5 0.  0.5 0. ]
  [0.  0.5 0.5 0. ]
  [0.  1.  0.  0. ]]

 [[1.  0.  0.  0. ]
  [0.5 0.  0.  0.5]
  [0.  0.5 0.  0.5]
  [0.  1.  0.  0. ]]

 [[0.5 0.  0.  0.5]
  [0.  0.  0.  1. ]
  [0.  0.  0.  1. ]
  [0.  0.  0.  0. ]]]

at each state, chosen action is :
[['T' 'Left' 'Left' 'Down']
 ['Up' 'Up' 'Down' 'Down']
 ['Up' 'Up' 'Down' 'Down']
 ['Up' 'Right' 'Right' 'T']]


# 2. Value Iteration

In [83]:
def policy_evaluation_for_value_iteration(grid_width, grid_height, action, policy, iter_num, reward=-1, dis=1):
    
    # table initialize
    post_value_table = np.zeros([grid_height, grid_width], dtype=float)
    
    # iteration
    if iter_num == 0:
        print(f'Iteration: {iter_num} \n{post_value_table}\n')
        return post_value_table
    
    for iteration in range(iter_num):
        next_value_table = np.zeros([grid_height, grid_width], dtype=float)
        for i in range(grid_height):
            for j in range(grid_width):
                if i == j and ((i == 0) or (i == 3)):
                    value_t = 0
                else :
                    value_t_list= []
                    for act in action:
                        i_, j_ = get_state([i,j], act)
                        value = (reward + dis*post_value_table[i_][j_])
                        value_t_list.append(value)
                    next_value_table[i][j] = max(value_t_list)
        iteration += 1
        
        # print result
        if (iteration % 10) != iter_num: 
            # print result 
            if iteration > 100 :
                if (iteration % 20) == 0: 
                    print(f'Iteration: {iteration} \n{next_value_table}\n')
            else :
                if (iteration % 10) == 0:
                    print(f'Iteration: {iteration} \n{next_value_table}\n')
        else :
            print(f'Iteration: {iteration} \n{next_value_table}\n')
        
       
        post_value_table = next_value_table
        
            
    return next_value_table

In [81]:
policy = reset_policy(grid_height, grid_width, action)

array([[[0.  , 0.  , 0.  , 0.  ],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.  , 0.  , 0.  , 0.  ]]])

In [84]:
value = policy_evaluation_for_value_iteration(grid_width, grid_height, action, policy, 1)
value = policy_evaluation_for_value_iteration(grid_width, grid_height, action, policy, 2)
value = policy_evaluation_for_value_iteration(grid_width, grid_height, action, policy, 3)
value = policy_evaluation_for_value_iteration(grid_width, grid_height, action, policy, 10)

Iteration: 1 
[[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1. -1.  0.]]

Iteration: 2 
[[ 0. -1. -2. -2.]
 [-1. -2. -2. -2.]
 [-2. -2. -2. -1.]
 [-2. -2. -1.  0.]]

Iteration: 3 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

Iteration: 10 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [86]:
value = policy_evaluation_for_value_iteration(grid_width, grid_height, action, policy, 100)

Iteration: 10 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

Iteration: 20 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

Iteration: 30 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

Iteration: 40 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

Iteration: 50 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

Iteration: 60 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

Iteration: 70 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

Iteration: 80 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

Iteration: 90 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

Iteration: 100 
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]

